In [13]:
# Paquetes
import pandas as pd
import numpy as np
import re

# Para cargar archivos de excel
import openpyxl

In [14]:
# Primero cargamos el archivo base de Excel y vemos las hojas que contiene
xls = pd.ExcelFile("Guapinol.xlsx")
hojas = xls.sheet_names

# Seleccionamos las hojas que necesitamos y las ponemos en una lista
hojasMinimos = [hoja for hoja in hojas if "MIN" in hoja]

# Eliminamos los espacios en blanco de los nombres de las hojas (keys del diccionario)
hojasMinimos = [i.strip() for i in hojasMinimos]

# Cargamos los datos de las hojas seleccionadas en un diccionario
datosMinimos = {}
for name in hojasMinimos:
    datosMinimos[name] = xls.parse(name)



In [15]:
primerHoja =next(iter(datosMinimos))
print(primerHoja)

GUAPINOL 77 78 MIN


In [16]:
# Para verificar el tipo de objeto en cada entrada del diccionario

type(datosMinimos[primerHoja])

pandas.core.frame.DataFrame

In [17]:
# Para acceder a cada hoja se usa el siguiente formato

datosMinimos[primerHoja].head(5)

,MES,DIA,ESC .,MC/SEG
0,JUL,16,0.24,3.61
1,AGO,1,0.25,3.93
2,SET,13,0.32,4.81
3,OCT,8,0.44,7.59
4,NOV,11,0.49,8.99


In [18]:
# Se busca primero el índice con la primer entrada no NaN
primerNoNaN = datosMinimos[primerHoja]['MC/SEG'].first_valid_index()

# Se eliminan las primeras entradas que son NaN

datosMinimos[primerHoja] = datosMinimos[primerHoja].iloc[primerNoNaN:]
datosMinimos[primerHoja].head(5)

,MES,DIA,ESC .,MC/SEG
0,JUL,16,0.24,3.61
1,AGO,1,0.25,3.93
2,SET,13,0.32,4.81
3,OCT,8,0.44,7.59
4,NOV,11,0.49,8.99


In [24]:
for name in hojasMinimos:
    # Elimina espacios en blanco en los nombres de las columnas
    datosMinimos[name].rename(columns=lambda x: x.strip(), inplace=True)

    # Renombramos columnas inconsistentes
    if 'ESC .' in datosMinimos[name].columns:
        datosMinimos[name].rename(columns={'ESC .': 'ESC'}, inplace=True)
    if 'MC/ S' in datosMinimos[name].columns:
        datosMinimos[name].rename(columns={'MC/ S': 'MC/SEG'}, inplace=True)
    
    # Eliminar columnas 'Unnamed:' si existen (como 'Unnamed: 4', etc.)
    datosMinimos[name] = datosMinimos[name].loc[:, ~datosMinimos[name].columns.str.contains('^Unnamed')]

    # Verifica las columnas
    print(f"Columnas en {name}: {list(datosMinimos[name].columns)}")

    # Verifica si tiene todas las columnas necesarias
    columnas_necesarias = {'MES', 'DIA', 'ESC', 'MC/SEG'}
    columnas_presentes = set(datosMinimos[name].columns)

    if not columnas_necesarias.issubset(columnas_presentes):
        print(f"Advertencia: {name} no tiene todas las columnas necesarias.")
        continue  # Saltar esta hoja si no tiene todas las columnas

    # Establece la lista de los meses correspondientes al primer año en el año hidrológico
    lista = datosMinimos[name]['MES'].isin(['MAY','JUN','JUL','AGO','SET','OCT','NOV','DIC'])

    # Se crea la columna Año
    datosMinimos[name].loc[:, 'ANO'] = '0'

    # Extrae el primer año de la hoja (ver código anterior para extracción correcta)
    match = re.search(r'(\d{2})\s+\d{2}', name)
    if not match:
        print(f"Error al extraer el año en el nombre de la hoja: {name}")
        continue

    year_base = '19' + match.group(1)  # Convertir a formato completo (ej. '1977')

    # Se establece el año según corresponda
    datosMinimos[name].loc[lista, 'ANO'] = year_base
    datosMinimos[name].loc[~lista, 'ANO'] = str(int(year_base) + 1)

    # Se reordenan las columnas correctamente
    datosMinimos[name] = datosMinimos[name][['DIA', 'MES', 'ANO', 'ESC', 'MC/SEG']]

# Verifica si la hoja principal existe antes de llamarla
if primerHoja in datosMinimos:
    print(datosMinimos[primerHoja].head(5))
else:
    print(f"La hoja {primerHoja} no existe en datosMinimos")


Columnas en GUAPINOL 77 78 MIN: ['MES', 'DIA', 'ESC', 'MC/SEG', 'ANO']
Columnas en GUAPINOL 78 79 MIN: ['DIA', 'MES', 'ANO', 'ESC', 'MC/SEG']
Columnas en GUAPINOL 78 80 MIN: ['MES', 'DIA', 'ESC', 'MC/SEC']
Advertencia: GUAPINOL 78 80 MIN no tiene todas las columnas necesarias.
Columnas en GUAPINOL 80 81 MIN: ['DIA', 'MES', 'ANO', 'ESC', 'MC/SEG']
Columnas en GUAPINOL 81 82 MIN: ['DIA', 'MES', 'ANO', 'ESC', 'MC/SEG']
Columnas en GUAPINOL 82 83 MIN: ['DIA', 'MES', 'ANO', 'ESC', 'MC/SEG']
Columnas en GUAPINOL 83 84 MIN: ['DIA', 'MES', 'ANO', 'ESC', 'MC/SEG']
Columnas en GUAPINOL 84 85 MIN: ['MES', 'DIA', 'ESC', 'MC/SEG']
Columnas en GUAPINOL 85 86 MIN: ['DIA', 'MES', 'ANO', 'ESC', 'MC/SEG']
Columnas en GUAPINOL 86 87 MIN: ['DIA', 'MES', 'ANO', 'ESC', 'MC/SEG']
Columnas en GUAPINOL 87 88 MIN: ['DIA', 'MES', 'ANO', 'ESC', 'MC/SEG']
   DIA  MES   ANO   ESC  MC/SEG
0   16  JUL  1977  0.24    3.61
1    1  AGO  1977  0.25    3.93
2   13  SET  1977  0.32    4.81
3    8  OCT  1977  0.44    7.59
4

In [30]:
datosMinimos[primerHoja].tail(40)

,DIA,MES,ANO,ESC,MC/SEG
0,16,JUL,1977,0.24,3.61
1,1,AGO,1977,0.25,3.93
2,13,SET,1977,0.32,4.81
3,8,OCT,1977,0.44,7.59
4,11,NOV,1977,0.49,8.99
5,31,DIC,1977,0.30,5.10
6,31,ENE,1978,0.22,3.23
7,22,FEB,1978,0.20,2.90
8,27,MAR,1978,0.17,2.37
9,28,ABR,1978,0.13,1.62


In [31]:
# Se juntan todas las entradas del diccionario
Minimos = pd.concat(datosMinimos.values(), ignore_index=True)

# Se verifica que el resultado sea un df
type(Minimos)

pandas.core.frame.DataFrame

In [33]:
Minimos.to_csv('NagatacMinimos.csv', index=False)